In [22]:
import json
import numpy as np
import pandas as pd
from dateutil import parser


In [23]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

### json 파일 읽어 dataframe으로 변환한다

In [24]:
# json 파일을 열고 각 줄을 개별적으로 파싱
data = []
with open('hopLogger.json', 'r') as file:
    for line in file:
        data.append(json.loads(line))

# 데이터를 DataFrame으로 변환
df = pd.DataFrame(data)
type(df)

pandas.core.frame.DataFrame

In [25]:
df

,Node,PerformanceInfo,level,msg,time
0,localhost:50001,"[{'BrokerId': 'localhost:50001', 'Cpu': '101.324860', 'Memory': '117477376', 'QueueLength': '0', 'QueueTime': '0.000000', 'ServiceTime': '765.910200', 'ResponseTime': '765.910200', 'InterArrivalTime': '9095.360300', 'Throughput': '1.305636', 'Timestamp': '2024-05-16 23:42:13.7603543 +0900 KST m=+18.862007301'}]",info,Advertisement(amazon > 50) a5f4b1aa-1e81-4d1e-a52c-a7b8b2ad96fa,2024-05-16 23:42:13
1,localhost:50001,"[{'BrokerId': 'localhost:50001', 'Cpu': '101.483465', 'Memory': '117624832', 'QueueLength': '0', 'QueueTime': '0.000000', 'ServiceTime': '854.678466', 'ResponseTime': '854.678466', 'InterArrivalTime': '2248.626800', 'Throughput': '1.170031', 'Timestamp': '2024-05-16 23:42:15.2550943 +0900 KST m=+20.356737701'}]",info,Publication(amazon = 120),2024-05-16 23:42:15
2,localhost:50002,"[{'BrokerId': 'localhost:50001', 'Cpu': '101.542867', 'Memory': '117383168', 'QueueLength': '0', 'QueueTime': '0.000000', 'ServiceTime': '0.000000', 'ResponseTime': '0.000000', 'InterArrivalTime': '0.000000', 'Throughput': '0.000000', 'Timestamp': '2024-05-17 13:49:58.379866 +0900 KST m=+4.857049301'}, {'BrokerId': 'localhost:50002', 'Cpu': '100.895866', 'Memory': '117964800', 'QueueLength': '1', 'QueueTime': '0.000000', 'ServiceTime': '0.000000', 'ResponseTime': '0.000000', 'InterArrivalTime': '491.675200', 'Throughput': '0.000000', 'Timestamp': '2024-05-17 13:49:59.8683132 +0900 KST m=+6.391723301'}]",info,Advertisement(nvidia > 70) e90d7af1-80e4-431f-8ad1-5b9a49f14c38,2024-05-17 13:49:59
3,localhost:50002,"[{'BrokerId': 'localhost:50003', 'Cpu': '100.786872', 'Memory': '117833728', 'QueueLength': '0', 'QueueTime': '0.000000', 'ServiceTime': '0.000000', 'ResponseTime': '0.000000', 'InterArrivalTime': '0.000000', 'Throughput': '0.000000', 'Timestamp': '2024-05-17 13:49:58.8695027 +0900 KST m=+5.143736001'}, {'BrokerId': 'localhost:50002', 'Cpu': '102.390140', 'Memory': '118259712', 'QueueLength': '0', 'QueueTime': '500.637500', 'ServiceTime': '1492.950200', 'ResponseTime': '1993.587700', 'InterArrivalTime': '491.675200', 'Throughput': '0.501608', 'Timestamp': '2024-05-17 13:49:59.9578701 +0900 KST m=+6.481280201'}]",info,Advertisement(nvidia > 50) a1382f53-511a-4c78-b394-1f16817b7988,2024-05-17 13:49:59
4,localhost:50002,"[{'BrokerId': 'localhost:50003', 'Cpu': '100.786872', 'Memory': '117833728', 'QueueLength': '0', 'QueueTime': '0.000000', 'ServiceTime': '0.000000', 'ResponseTime': '0.000000', 'InterArrivalTime': '0.000000', 'Throughput': '0.000000', 'Timestamp': '2024-05-17 13:49:58.8695027 +0900 KST m=+5.143736001'}, {'BrokerId': 'localhost:50006', 'Cpu': '101.176093', 'Memory': '117862400', 'QueueLength': '2', 'QueueTime': '0.000000', 'ServiceTime': '0.000000', 'ResponseTime': '0.000000', 'InterArrivalTime': '91.557800', 'Throughput': '0.000000', 'Timestamp': '2024-05-17 13:50:00.4882064 +0900 KST m=+6.606637701'}, {'BrokerId': 'localhost:50002', 'Cpu': '102.897484', 'Memory': '117952512', 'QueueLength': '1', 'QueueTime': '333.758333', 'ServiceTime': '794.005200', 'ResponseTime': '1127.763533', 'InterArrivalTime': '61.669400', 'Throughput': '0.886711', 'Timestamp': '2024-05-17 13:50:01.4291337 +0900 KST m=+7.952543801'}]",info,Advertisement(nvidia > 50) a1382f53-511a-4c78-b394-1f16817b7988,2024-05-17 13:50:01


In [26]:
type(df['PerformanceInfo'])

pandas.core.series.Series

### PerformanceInfo 컬럼의 트랜잭션을 홉별로 컬럼화 한다
### 이때, 가장 긴 홉 수에 맞춰 짧은 홉은 np.Nan으로 채운다

In [27]:
# PerformanceInfo 컬럼 파싱
# 데이터를 파싱하고, 각 트랜잭션의 모든 홉을 하나의 행으로 변환

max_hops = 0
rows = []
keys = set()

for tran in df['PerformanceInfo']:
    # tran이 문자열인지 확인하고, 맞다면 json.loads를 사용
    parsed = json.loads(tran) if isinstance(tran, str) else tran
    for hop in parsed:
        keys.update(hop.keys())
# print(keys)

for tran in df['PerformanceInfo']:
    # tran이 문자열인지 확인하고, 맞다면 json.loads를 사용
    parsed = json.loads(tran) if isinstance(tran, str) else tran
    row = {}
    for idx, hop in enumerate(parsed):
        for key, value in hop.items():
            row[f'{key}_{idx}'] = value
    rows.append(row)
    max_hops = max(max_hops, len(parsed))

# 패딩: 모든 행을 가장 많은 홉 수에 맞추어 확장
final_rows = []
for row in rows:
    padded_row = {f'{key}_{i}': row.get(f'{key}_{i}', np.nan) for i in range(max_hops) for key in keys}
    final_rows.append(padded_row)

# 최종 DataFrame 생성
performance_df = pd.DataFrame(final_rows)

# 원래 DataFrame과 합치기
result_df = pd.concat([df[['Node', 'level', 'msg', 'time']], performance_df], axis=1)

# # 결과 확인
# print(result_df)


In [28]:
result_df

,Node,level,msg,time,QueueTime_0,BrokerId_0,QueueLength_0,ResponseTime_0,ServiceTime_0,Throughput_0,InterArrivalTime_0,Cpu_0,Timestamp_0,Memory_0,QueueTime_1,BrokerId_1,QueueLength_1,ResponseTime_1,ServiceTime_1,Throughput_1,InterArrivalTime_1,Cpu_1,Timestamp_1,Memory_1,QueueTime_2,BrokerId_2,QueueLength_2,ResponseTime_2,ServiceTime_2,Throughput_2,InterArrivalTime_2,Cpu_2,Timestamp_2,Memory_2
0,localhost:50001,info,Advertisement(amazon > 50) a5f4b1aa-1e81-4d1e-a52c-a7b8b2ad96fa,2024-05-16 23:42:13,0.000000,localhost:50001,0,765.910200,765.910200,1.305636,9095.360300,101.324860,2024-05-16 23:42:13.7603543 +0900 KST m=+18.862007301,117477376,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,localhost:50001,info,Publication(amazon = 120),2024-05-16 23:42:15,0.000000,localhost:50001,0,854.678466,854.678466,1.170031,2248.626800,101.483465,2024-05-16 23:42:15.2550943 +0900 KST m=+20.356737701,117624832,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,localhost:50002,info,Advertisement(nvidia > 70) e90d7af1-80e4-431f-8ad1-5b9a49f14c38,2024-05-17 13:49:59,0.000000,localhost:50001,0,0.000000,0.000000,0.000000,0.000000,101.542867,2024-05-17 13:49:58.379866 +0900 KST m=+4.857049301,117383168,0.000000,localhost:50002,1,0.000000,0.000000,0.000000,491.675200,100.895866,2024-05-17 13:49:59.8683132 +0900 KST m=+6.391723301,117964800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,localhost:50002,info,Advertisement(nvidia > 50) a1382f53-511a-4c78-b394-1f16817b7988,2024-05-17 13:49:59,0.000000,localhost:50003,0,0.000000,0.000000,0.000000,0.000000,100.786872,2024-05-17 13:49:58.8695027 +0900 KST m=+5.143736001,117833728,500.637500,localhost:50002,0,1993.587700,1492.950200,0.501608,491.675200,102.390140,2024-05-17 13:49:59.9578701 +0900 KST m=+6.481280201,118259712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,localhost:50002,info,Advertisement(nvidia > 50) a1382f53-511a-4c78-b394-1f16817b7988,2024-05-17 13:50:01,0.000000,localhost:50003,0,0.000000,0.000000,0.000000,0.000000,100.786872,2024-05-17 13:49:58.8695027 +0900 KST m=+5.143736001,117833728,0.000000,localhost:50006,2,0.000000,0.000000,0.000000,91.557800,101.176093,2024-05-17 13:50:00.4882064 +0900 KST m=+6.606637701,117862400,333.758333,localhost:50002,1,1127.763533,794.005200,0.886711,61.669400,102.897484,2024-05-17 13:50:01.4291337 +0900 KST m=+7.952543801,117952512


In [29]:
# # df의 PerformanceInfo 열을 파싱
# df['PerformanceInfo'] = df['PerformanceInfo'].apply(lambda x: json.loads(x))
